In [1]:
from iroha import Iroha, IrohaCrypto, IrohaGrpc, primitive_pb2
import ed25519

# Connection information to iroha
net = IrohaGrpc("127.0.0.1:50051")
admin = Iroha("admin@test")

In [ ]:


# Creating a key pair
signing_key, verifying_key = ed25519.create_keypair()

# Save the created key pair
open("iroha@test.prib","wb").write(signing_key.to_ascii(encoding="hex"))
open("iroha@test.pub","wb").write(verifying_key.to_ascii(encoding="hex"))

# Convert from binary to hexadecimal
vkey_hex = verifying_key.to_ascii(encoding="hex")

# Creating a Transaction
transfer_tx = admin.transaction(
 [admin.command(
    'CreateAccount',
    account_name ='iroha',
    domain_id = 'test',
    public_key = vkey_hex
 )]
)

# Transaction signature
IrohaCrypto.sign_transaction(transfer_tx,"f101537e319568c765b2cc89698325604991dca57b9716b58016b253506cab70")

# Send Transaction
net.send_tx(transfer_tx)

# Check the result
for status in net.tx_status_stream(transfer_tx):
    print(status)

In [7]:

get_block_query = admin.query(
        'GetAccount',
        account_id = 'iroha@test'  
)

# Sign Query
IrohaCrypto.sign_query(get_block_query, "f101537e319568c765b2cc89698325604991dca57b9716b58016b253506cab70")

# Send Query
response = net.send_query(get_block_query)

# Response output
print(response)
    

account_response {
  account {
    account_id: "iroha@test"
    domain_id: "test"
    quorum: 1
    json_data: "{}"
  }
  account_roles: "user"
}
query_hash: "bc6970c923512119aac0d62beeeb6988e8279c43a3f50594bc7c5669bb7b1b6c"



In [ ]:
iroha = Iroha("iroha@test")
# add peer to network
def add_peer():
    peer1 = primitive_pb2.Peer()
    peer1.address = '192.168.10.49:50051'
    peer1.peer_key = '2da10813a07da65746f86f8de3ab2640c87c8cc1af11321f37320561535e8317'
    tx = iroha.transaction([
            iroha.command('AddPeer', peer=peer1)
    ], creator_account="admin@test", quorum=1)

    IrohaCrypto.sign_transaction(tx, "f101537e319568c765b2cc89698325604991dca57b9716b58016b253506cab70")
    net.send_tx(tx)
    for status in net.tx_status_stream(tx):
        print(status)
add_peer()

In [21]:
#iroha.command('AddPeer', peer=peer)
help(iroha)

Help on Iroha in module iroha.iroha object:

class Iroha(builtins.object)
 |  Iroha(creator_account=None)
 |  
 |  Collection of factory methods for transactions and queries creation
 |  
 |  Methods defined here:
 |  
 |  __init__(self, creator_account=None)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  blocks_query(self, counter=1, creator_account=None, created_time=None)
 |      Creates a protobuf query for a blocks stream
 |      :param counter: query counter, should be incremented for each new query
 |      :param creator_account: account id of query creator
 |      :param created_time: query creation timestamp in milliseconds
 |      :return: a proto blocks query
 |  
 |  query(self, name, counter=1, creator_account=None, created_time=None, page_size=None, first_tx_hash=None, **kwargs)
 |      Creates a protobuf query with specified set of entities
 |      :param name: CamelCased name of query to be executed
 |      :param counter: query counter,

In [ ]:
def create_domain():
    cmds = [
        admin.command("CreateDomain", domain_id="chain", default_role="admin")
    ]

    tx = admin.transaction(cmds,creator_account="admin@test")
    IrohaCrypto.sign_transaction(tx, "f101537e319568c765b2cc89698325604991dca57b9716b58016b253506cab70")
    net.send_tx(tx)
    for status in net.tx_status_stream(tx):
        print(status)
    return tx
create_domain()

In [40]:
def query_pending_tx():
    tx = admin.query("GetPendingTransactions")
    IrohaCrypto.sign_query(tx, "f101537e319568c765b2cc89698325604991dca57b9716b58016b253506cab70")
    response = net.send_query(tx)
    print(response)
query_pending_tx()

transactions_response {
}
query_hash: "c56d19343f5f67b2ccd1af5ec220b6f979a67c7fe9b3066d0984d1f322d30aa9"



In [42]:
def remove_peer():
    tx = admin.transaction([admin.command("RemovePeer", public_key="2da10813a07da65746f86f8de3ab2640c87c8cc1af11321f37320561535e8317")])
    IrohaCrypto.sign_transaction(tx, "f101537e319568c765b2cc89698325604991dca57b9716b58016b253506cab70")
    net.send_tx(tx)
    for status in net.tx_status_stream(tx):
        print(status)
remove_peer()
    

('ENOUGH_SIGNATURES_COLLECTED', 9, 0)


KeyboardInterrupt: 